# 🚀 Free FastAPI Deployment - 20250804_124154

**Repository:** https://github.com/sonagara-vashram/test.git

**Status:** Ready for deployment

---

## 📋 Instructions:
1. Run all cells in sequence (Runtime → Run all)
2. Wait for deployment to complete (~2-3 minutes)
3. Copy your public API URL from the output
4. Keep this notebook running to maintain the API

---

## ⚡ Quick Deploy:
Click **Runtime → Run all** to auto-deploy your FastAPI backend!

In [ ]:
# 📦 Installing required packages
print('🔧 Installing packages...')
!pip install -q pyngrok fastapi uvicorn[standard] python-dotenv requests aiofiles fastapi==0.104.1 uvicorn==0.24.0

print('✅ All packages installed successfully!')
print('📦 Installed packages:')
packages = ['pyngrok', 'fastapi', 'uvicorn[standard]', 'python-dotenv', 'requests', 'aiofiles', 'fastapi==0.104.1', 'uvicorn==0.24.0']
for pkg in packages[:5]:  # Show first 5
    print(f'  ✓ {pkg}')
if len(packages) > 5:
    print(f'  ... and {len(packages) - 5} more packages')

In [ ]:
# 📁 Cloning repository
import os
print('📥 Cloning repository...')

# Remove existing directory if present
if os.path.exists('/content/app'):
    !rm -rf /content/app

!git clone https://github.com/sonagara-vashram/test.git /content/app

# Change to app directory
os.chdir('/content/app')

print('✅ Repository cloned successfully!')
print('📂 Repository contents:')
!ls -la

# Install project requirements if exists
if os.path.exists('requirements.txt'):
    print('\n📋 Installing project requirements...')
    !pip install -q -r requirements.txt
    print('✅ Project requirements installed!')
else:
    print('\n⚠️ No requirements.txt found in repository')

In [ ]:
# 🔐 Setting up environment variables
import os

os.environ["SECRET_KEY"] = "your_secret_key"

print('✅ Set 1 environment variables')
print('🔐 Environment variables configured:')
for key in os.environ.keys():
    if key in ['SECRET_KEY']:
        print(f'  ✓ {key}')

In [ ]:
# 🔍 FastAPI App Detection and Startup
import threading
import time
import uvicorn
import importlib.util
import sys

app_file = 'main.py'
app_variable = 'app'

print(f'🔍 Loading FastAPI app from {app_file}...')

# Import the module
try:
    spec = importlib.util.spec_from_file_location('main_app', app_file)
    app_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(app_module)
    
    # Get the FastAPI app instance
    if hasattr(app_module, app_variable):
        fastapi_app = getattr(app_module, app_variable)
        print(f'✅ Found FastAPI app: {app_variable}')
        print(f'📊 App type: {type(fastapi_app)}')
    else:
        print(f'❌ App variable "{app_variable}" not found in {app_file}')
        print('📋 Available variables:')
        for attr in dir(app_module):
            if not attr.startswith('_'):
                print(f'  - {attr}: {type(getattr(app_module, attr))}')
        raise Exception(f'App variable not found')
    
    # Start FastAPI server in background
    def start_server():
        uvicorn.run(fastapi_app, host='0.0.0.0', port=8000, log_level='warning')
    
    print('🚀 Starting FastAPI server...')
    server_thread = threading.Thread(target=start_server, daemon=True)
    server_thread.start()
    
    # Wait for server to start
    time.sleep(8)
    print('✅ FastAPI server started on port 8000!')
    
except Exception as e:
    print(f'❌ Error starting FastAPI app: {e}')
    print('\n🔧 Troubleshooting tips:')
    print('1. Check if your FastAPI app variable name is correct')
    print('2. Ensure your app file has no syntax errors')
    print('3. Verify all dependencies are installed')
    raise

In [ ]:
# 🌐 Creating Public API with ngrok
from pyngrok import ngrok
import requests
import time
from google.colab import userdata

print('🔗 Creating ngrok tunnel...')

try:
    # Set ngrok authtoken
    # You need to get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
    # Add it to Colab secrets with the name 'NGROK_AUTH_TOKEN'
    try:
        ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
        ngrok.set_auth_token(ngrok_auth_token)
        print('✅ ngrok authtoken retrieved from Colab secrets!')
    except:
        # Fallback: Use default free ngrok (limited sessions)
        print('⚠️ NGROK_AUTH_TOKEN not found in Colab secrets.')
        print('💡 Using free ngrok (sessions may be limited)')
        print('📋 To get unlimited sessions:')
        print('   1. Sign up at https://ngrok.com (free)')
        print('   2. Get your auth token from dashboard')
        print('   3. Add it to Colab secrets as NGROK_AUTH_TOKEN')

    # Create HTTP tunnel to port 8000
    public_tunnel = ngrok.connect(8000)
    public_url = public_tunnel.public_url
    
    print('\n🎉 DEPLOYMENT SUCCESSFUL! 🎉')
    print('=' * 60)
    print(f'🔗 Public API URL: {public_url}')
    print(f'📚 Swagger UI: {public_url}/docs')
    print(f'📋 ReDoc: {public_url}/redoc')
    print('=' * 60)
    
    # Test the API
    print('\n🧪 Testing API connection...')
    try:
        response = requests.get(f'{public_url}/', timeout=10)
        if response.status_code == 200:
            print('✅ API is responding correctly!')
            print(f'📊 Response: {response.text[:100]}...')
        else:
            print(f'⚠️ API returned status code: {response.status_code}')
    except Exception as test_error:
        print(f'⚠️ API test failed: {test_error}')
    
    print('\n🎯 Your FastAPI backend is now live and accessible worldwide!')
    print('💡 Copy the Public API URL above to use in your frontend')
    print('⚠️ Keep this notebook running to maintain the API')
    print('\n📱 Quick Links:')
    print(f'   🏠 Homepage: {public_url}/')
    print(f'   📚 API Docs: {public_url}/docs')
    print(f'   📋 ReDoc: {public_url}/redoc')
    
    # Store URL for monitoring
    globals()['PUBLIC_API_URL'] = public_url
    
except Exception as e:
    print(f'❌ Failed to create ngrok tunnel: {e}')
    print('\n🔧 Troubleshooting:')
    print('1. Ensure FastAPI server is running (check previous cell)')
    print('2. Wait a moment and try running this cell again')
    print('3. Check if port 8000 is accessible')
    print('4. For unlimited ngrok sessions, add NGROK_AUTH_TOKEN to Colab secrets')
    print('   📋 Get token from: https://dashboard.ngrok.com/get-started/your-authtoken')
    raise

In [ ]:
# 📊 API Health Monitoring
import requests
import time
from datetime import datetime

# Get the public URL from previous cell
if 'PUBLIC_API_URL' in globals():
    api_url = PUBLIC_API_URL
    print(f'🔄 Monitoring API: {api_url}')
    print('\n📊 Health Check Status:')
    print('Press Ctrl+C to stop monitoring\n')
    
    try:
        check_count = 0
        while True:
            try:
                start_time = time.time()
                response = requests.get(f'{api_url}/', timeout=5)
                response_time = round((time.time() - start_time) * 1000, 2)
                
                timestamp = datetime.now().strftime('%H:%M:%S')
                
                if response.status_code == 200:
                    print(f'✅ {timestamp} - API healthy (Status: {response.status_code}, Response: {response_time}ms)')
                else:
                    print(f'⚠️ {timestamp} - API warning (Status: {response.status_code})')
                
                check_count += 1
                if check_count % 10 == 0:
                    print(f'\n📈 Completed {check_count} health checks')
                    print(f'🌐 Your API: {api_url}')
                    print(f'📚 Swagger: {api_url}/docs\n')
                
            except requests.RequestException as e:
                timestamp = datetime.now().strftime('%H:%M:%S')
                print(f'❌ {timestamp} - API unreachable: {e}')
            
            time.sleep(30)  # Check every 30 seconds
            
    except KeyboardInterrupt:
        print('\n🛑 Monitoring stopped by user')
        print(f'📊 Total health checks performed: {check_count}')
        print(f'🌐 Your API is still running at: {api_url}')
        
else:
    print('❌ No API URL found. Please run the ngrok cell first.')